## 로지스틱회귀모델 with Word2Vec

### 라이브러리 불러오기

In [1]:
import os
import re
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

### 학습 데이타 불러오기: 전처리가 된 텍스트 파일
- wrod2vec은 단어로 표현된 리스트를 입력값으로 넣어야 하기 때문데 
- 전처리된 텍스트 csv 파일을 불러온 후 각 단어들의 리스트로 나눠야 함

In [2]:
DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [3]:
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

In [4]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

### word2vec을 사용하기 위해 입력값을 단어로 구분된 리스트로 만들어야 한다.
- 읽어온 학습 데이타 전체 리뷰를 단어 리스트로 바꾼다.
- 각 리뷰를 split 함수를 이용해서 띄어쓰기 기준으로 구분한 후 리스트에 하나씩 추가해서 단어리스트를 만든다.

In [5]:
sentences = []
for review in reviews:
    sentences.append(review.split())

In [7]:
sentences[:1]

[['stuff',
  'going',
  'moment',
  'mj',
  'started',
  'listening',
  'music',
  'watching',
  'odd',
  'documentary',
  'watched',
  'wiz',
  'watched',
  'moonwalker',
  'maybe',
  'want',
  'get',
  'certain',
  'insight',
  'guy',
  'thought',
  'really',
  'cool',
  'eighties',
  'maybe',
  'make',
  'mind',
  'whether',
  'guilty',
  'innocent',
  'moonwalker',
  'part',
  'biography',
  'part',
  'feature',
  'film',
  'remember',
  'going',
  'see',
  'cinema',
  'originally',
  'released',
  'subtle',
  'messages',
  'mj',
  'feeling',
  'towards',
  'press',
  'also',
  'obvious',
  'message',
  'drugs',
  'bad',
  'kay',
  'visually',
  'impressive',
  'course',
  'michael',
  'jackson',
  'unless',
  'remotely',
  'like',
  'mj',
  'anyway',
  'going',
  'hate',
  'find',
  'boring',
  'may',
  'call',
  'mj',
  'egotist',
  'consenting',
  'making',
  'movie',
  'mj',
  'fans',
  'would',
  'say',
  'made',
  'fans',
  'true',
  'really',
  'nice',
  'actual',
  'feature

### Word2Vec 벡터화

#### Word2Vec 모델의 하이퍼파라미터 설정

- num_features : 각 단어에 대한 임베딩된 벡터의 차원 설정
- min_word_count : 모델에 의미 있는 단어를 가지고 학습하기 위해 적은 빈도수의 단어들은 학습하지 않는다.
- num_workers : 모델 학습 시 학습을 위한 프로세스의 개수를 지정
- context : word2vec을 수행하기 위한 컨텍스트 윈도우 크기를 지정
- downsampling : word2vec 학습을 수행할 때 빠른 학습을 위해 정답 단어 라벨에 대한 다운 샘플링 비율을 지정
    - 보통 0.001이 좋은 성능을 낸다고 하낟.

In [8]:
num_features = 300    # 워드 벡터 특징값 수
min_word_count = 40   # 단어데 대한 최소 빈도수 
num_workers = 4       # 프로세스 개수
context = 10          # 컨텍스트 윈도 크기
downsampling = 1e-3   # 다운 샘플링 비율

#### 설정된 하이퍼파라미터를 가지고 word2vec 학습
- gensim 라이브러리 사용, gensim.models에 있는 word2vec 모듈 사용

In [ ]:
#!pip install gensim

- word2vec 학습하는 과정에서 진행 상황을 확인하기 위해 logging 사용
- level(로그 수준)를 INFO에 맞추면 word2vec 학습과정에서 로그메세지를 양식에 맞게 INFO 수준으로 보여줌

In [9]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
   level=logging.INFO)

### Word2Vec 학습
- word2vec 모듈에 있는 Word2Vec 객체를 생성하여 실행
- 학습하고 생성된 객체는 model 변수에 할당
- 입력데이타와 하이퍼파라미터를 순서대로 입력하여 학습 실행

In [11]:
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2021-03-04 11:20:15,495 : INFO : collecting all words and their counts
2021-03-04 11:20:15,499 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-03-04 11:20:16,015 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2021-03-04 11:20:16,520 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2021-03-04 11:20:16,714 : INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2021-03-04 11:20:16,715 : INFO : Loading a fresh vocabulary
2021-03-04 11:20:16,808 : INFO : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2021-03-04 11:20:16,808 : INFO : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2021-03-04 11:20:16,867 : INFO : deleting the raw counts dictionary of 74065 items
2021-03-04 11:20:16,867 : INFO : sample=0.001 downsamples 30 most-common words
2021-03-04 11:20:16,882 :

### word2vec으로 학습된 모델 저장
- 모델 저장 model.save(모델이름) 
- 저장한 모델은 Word2Vec.load()로 불러올 수 있다.

In [12]:
model_name = "./data_out/300features_40minwords_10context"
model.save(model_name)

2021-03-04 11:23:48,353 : INFO : saving Word2Vec object under ./data_out/300features_40minwords_10context, separately None
2021-03-04 11:23:48,354 : INFO : not storing attribute vectors_norm
2021-03-04 11:23:48,354 : INFO : not storing attribute cum_table
2021-03-04 11:23:48,780 : INFO : saved ./data_out/300features_40minwords_10context


### 로지스틱 회귀 모델 적용하기 위해 전처리
- 하나의 리뷰를 같은 형태의 입력값으로 만들어야 한다.
- word2vec 모델에서 각 단어가 벡터로 표현되어 있고, 리뷰마다 단어의 개수가 모두 다르기 때문에 입력값을 하나의 형태로 만들어야 한다.
- 가장 단순한 방법 : 문장에 있는 모든 단어의 벡터값에 대해 평균을 내서 리뷰 하나당 하나의 벡터를 만드는 것

### 하나의 리뷰에 대해 전체 단어의 평균값을 계산하는 함수
- 문장에 특징값을 만들 수 있는 함수
- get_features(단어의 모음인 하나의 리뷰, word2vec모델, word2vec으로 임베딩할 때 정했던 벡터의 차원수)

In [13]:
def get_features(words, model, num_features):
    
    # np.zeros 함수를 사용해 모두 0의 값을 가지는 벡터 생성
    feature_vector = np.zeros((num_features),dtype=np.float32)

    # 문장이 단어가 해당 모델 단어 사전에 속하는지 확인하기 위해 set 객체로 생성
    num_words = 0
    index2word_set = set(model.wv.index2word)

    # 리뷰를 구성하는 단어에 대해 임베딩된 벡터가 있는 단어 벡터의 합을 구한다.
    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])

    # 사용한 단어의 전체 개수로 나눠서 평균 벡터의 값을 구한다.
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

### 전체 리뷰에 대해 각 리뷰의 평균 벡터를 구하는 함수
- get_dataset(학습데이타의 전체 리뷰데이타, word2vec모델, word2vec으로 임베딩할때 정했던 벡터의 차원수)

In [14]:
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

### 전체 리뷰에 대해 평균 벡터 구하기

In [15]:
test_data_vecs = get_dataset(sentences, model, num_features)

c:\users\jinsuk kim\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [16]:
test_data_vecs[:1]

array([[-1.00970184e-02, -6.70368820e-02,  5.29728085e-02,
        -2.06597373e-02,  1.43107489e-01,  2.62437314e-01,
         1.10805079e-01,  1.08345069e-01,  1.21532241e-02,
        -1.30330294e-01,  1.15363471e-01, -2.41566628e-01,
         3.98410857e-02, -3.14722359e-01,  1.60251498e-01,
         4.45713811e-02,  9.25093237e-03,  7.60926977e-02,
        -1.76173285e-01, -2.33845741e-01,  6.36781454e-02,
         3.10567953e-03,  4.58594933e-02, -3.45952883e-02,
        -5.00632487e-02,  6.29462972e-02,  1.31065175e-01,
        -1.88865792e-02, -1.81590754e-03, -1.89025894e-01,
        -1.21760480e-01,  1.64418355e-01, -3.23844284e-01,
         9.73522514e-02,  1.10095670e-03,  1.44492805e-01,
        -7.20995246e-03, -7.71756396e-02,  3.38824093e-02,
        -7.24842176e-02,  1.57285526e-01, -5.89011125e-02,
         3.37456097e-03,  3.62957940e-02, -2.89626479e-01,
         1.42615542e-01, -1.85335234e-01, -1.67775080e-02,
         1.44413039e-01, -6.17982773e-03,  2.15693265e-0

### 학습 데이타와 테스트 데이타 분리

In [17]:
from sklearn.model_selection import train_test_split
import numpy as np

# RANDOM_SEED = 42
# TEST_SPLIT = 0.2

X = test_data_vecs
y = np.array(sentiments)

X_train, X_test, y_train, y_test = train_test_split(X, y, \
                                                    test_size=TEST_SPLIT, random_state=RANDOM_SEED)

### 로지스틱 회귀 모델 생성

In [19]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')

### 모델 학습

In [20]:
lgs.fit(X_train, y_train)

C:\Users\Jinsuk Kim\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### 모델 예측 및 평가

In [22]:
# 예측
predicted = lgs.predict(X_test)
predicted

array([0, 1, 0, ..., 0, 0, 0])

In [26]:
# 실제 정답
y_train

array([0, 0, 0, ..., 1, 1, 0])

In [21]:
# 정확도
print("Accuracy: %f" % lgs.score(X_test, y_test)) 

Accuracy: 0.862400


- 해석)
- 분류 정확도가 86% 정도
- TF-IDF는 85% 정도이므로 TF-IDF에 비해 Word2Vec이 성능이 다소 좋다고 볼 수 있음

### 생성된 로지스틱 회귀모델을 이용하여 평가 데이타 결과를 예측

- 평가 데이타 파일 불러오기: 텍스트 형태 파일

In [30]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_review = list(test_data['review'])

In [31]:
test_data.head(5)

,review,id
0,naturally film main themes mortality nostalgia...,"""12311_10"""
1,movie disaster within disaster film full great...,"""8348_2"""
2,movie kids saw tonight child loved one point k...,"""5828_4"""
3,afraid dark left impression several different ...,"""7186_2"""
4,accurate depiction small time mob life filmed ...,"""12128_7"""


- 평가 데이타도 학습 데이타와 동일하게 각 리뷰가 하나의 문자열로 이루어져 있으므로 
- 평가 데이타도 각 단어의 리스트로 만들어야 한다.

In [32]:
test_sentences = list()
for review in test_review:
    test_sentences.append(review.split())

- 평가 데이타도 word2vec으로 임베딩된 벡터값으로 만들어야 한다.
- 평가 데이타에는 새롭게 word2vec 모델을 만들면 안되고, 학습 데이타 모델에 학습시킨 모델을 이용하여
- 평가 데이타에 각 단어들을 벡터로 만들어 각 리뷰에 대한 특징값을 만든다.
- 그런 후 이전에 정의했던 함수를 동일하게 적용한다.

In [33]:
test_data_vecs = get_dataset(test_sentences, model, num_features)

c:\users\jinsuk kim\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


- 생성된 로지스틱회귀모델을 이용하여 예측

In [35]:
test_predicted = lgs.predict(test_data_vecs)
print(test_predicted)

[1 0 1 ... 0 1 1]


- 예측한 결과값을 데이타프레임형태로 만들어 csv 파일로 저장
- 각 데이타의 고유값 id와 결과값으로 구성
- 캐글에 csv 파일 제출한 후 정확도 확인

In [36]:
DATA_OUT_PATH = './data_out/'

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
ids = list(test_data['id'])
answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_w2v_answer.csv', index=False, quoting=3)